## Assignment 2.1: Text classification via RNN (50 points)

In this assignment you will perform sentiment analysis of the IMDBs reviews by using RNN.

In [ ]:
!pip install torch==1.6.0
!pip install torchtext==0.7
!pip install numpy
!pip install pandas

     |████████████████████████████████| 748.8 MB 17 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.
     |████████████████████████████████| 4.5 MB 25.9 MB/s 
     |████████████████████████████████| 1.2 MB 47.5 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [ ]:
!pip install torch

In [ ]:
!pip install torchtext

In [ ]:
!pip install torch==1.11.0
!pip install torchtext==0.12.0


     |████████████████████████████████| 750.6 MB 12 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.6.0
    Uninstalling torch-1.6.0:
      Successfully uninstalled torch-1.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.11.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.11.0 which is incompatible.
     |████████████████████████████████| 10.4 MB 22.9 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.7.0
    Uninstalling torchtext-0.7.0:
      Successfully uninstalled torchtext-0.7.0


In [ ]:
!pip install torchtext

In [ ]:
!pip install -U torch==1.8.0 torchtext==0.9.0

# Reload environment
#exit()

     |████████████████████████████████| 735.5 MB 14 kB/s 
     |████████████████████████████████| 7.1 MB 38.8 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.8.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.8.0 which is incompatible.


In [ ]:
import torchtext
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

ОБновите список библиотек, зависисмоти неправильные были изначально Проверить можно тут - https://github.com/pytorch/text/releases/tag/v0.9.0-rc5

In [ ]:
import pandas as pd
import numpy as np
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

sns.set(style='darkgrid', font_scale=1.3)

In [ ]:
from torchtext.legacy.data import Field, LabelField
from torchtext.legacy.data import BucketIterator

### Preparing Data

In [ ]:
TEXT = Field(sequential=True, lower=True)
LABEL = LabelField()

In [ ]:
train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split()

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:04<00:00, 18.4MB/s]


In [ ]:
%%time
TEXT.build_vocab(trn)

CPU times: user 1.37 s, sys: 57 ms, total: 1.43 s
Wall time: 1.43 s


In [ ]:
LABEL.build_vocab(trn)

In [ ]:
TEXT.vocab.freqs.most_common(10)

[('the', 224622),
 ('a', 111677),
 ('and', 110488),
 ('of', 100561),
 ('to', 93661),
 ('is', 72691),
 ('in', 63181),
 ('i', 49112),
 ('this', 48835),
 ('that', 46709)]

### Creating the Iterator

During training, we'll be using a special kind of Iterator, called the **BucketIterator**. When we pass data into a neural network, we want the data to be padded to be the same length so that we can process them in batch:

e.g.
\[ 
\[3, 15, 2, 7\],
\[4, 1\], 
\[5, 5, 6, 8, 1\] 
\] -> \[ 
\[3, 15, 2, 7, **0**\],
\[4, 1, **0**, **0**, **0**\], 
\[5, 5, 6, 8, 1\] 
\] 

If the sequences differ greatly in length, the padding will consume a lot of wasteful memory and time. The BucketIterator groups sequences of similar lengths together for each batch to minimize padding.

Complete the definition of the **BucketIterator** object

In [ ]:
train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(64, 64, 64),
        sort=True,
        sort_key=lambda x: len(x.text),
        sort_within_batch=False,
        device='cuda',
        repeat=False
)

In [ ]:
len(TEXT.vocab)

Let's take a look at what the output of the BucketIterator looks like. Do not be suprised **batch_first=True**

In [ ]:
batch = next(train_iter.__iter__()); batch.text

The batch has all the fields we passed to the Dataset as attributes. The batch data can be accessed through the attribute with the same name.

In [ ]:
batch.__dict__.keys()

### Define the RNN-based text classification model (20 points)

Start simple first. Implement the model according to the shema below.  
![alt text](https://miro.medium.com/max/1396/1*v-tLYQCsni550A-hznS0mw.jpeg)


In [ ]:
class RNNBaseline(nn.Module):
    def __init__(self, hidden_dim, emb_dim):
        super().__init__()
        self.emb = nn.Embedding(len(TEXT.vocab), emb_dim)
        self.rnn = nn.GRU(emb_dim, hidden_dim, batch_first=False)
        self.linear = nn.Linear(hidden_dim, 1)
            
    def forward(self, seq):
        embeddings = self.emb(seq)
        last_hidden = self.rnn(embeddings)[1]
        # preds are outputs of linear layer BEFORE sigmoid
        preds = self.linear(last_hidden).reshape(-1)
        return preds

NameError: name 'nn' is not defined

In [ ]:
em_sz = 200
nh = 300
model = RNNBaseline(nh, emb_dim=em_sz); model

If you're using a GPU, remember to call model.cuda() to move your model to the GPU.

In [ ]:
model.cuda()

### The training loop (10 points)

Define the optimization and the loss functions.

In [ ]:
learning_rate = 1e-3
opt = optim.Adam(model.parameters(), lr=learning_rate)
loss_func = nn.BCEWithLogitsLoss()

Define the stopping criteria.

In [ ]:
epochs = 25

In [ ]:
%%time
for epoch in range(1, epochs + 1):
    running_loss = 0.0
    running_corrects = 0
    model.train() 
    for batch in train_iter: 
        
        x = batch.text
        y = batch.label

        opt.zero_grad()
        preds = model(x)   
        loss = loss_func(preds, y)
        loss.backward()
        opt.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(trn)
    
    val_loss = 0.0
    model.eval()
    for batch in val_iter:
        
        x = batch.text
        y = batch.label
        
        preds = model(x) 
        loss = loss_func(preds, y)
        val_loss += loss.item()
        
    val_loss /= len(vld)
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, epoch_loss, val_loss))

### Calculate performance of the trained model (10 points)

In [ ]:
for batch in test_iter:
    x = batch.text
    y = batch.label

Write down the calculated performance

### Accuracy:
### Precision:
### Recall:
### F1:

In [ ]:
y_pred = []
y_true = []
model.eval()

for batch in test_iter:
    x = batch.text
    y = batch.label

    y_pred += list((torch.sigmoid(model(x)) >= 0.5).detach().cpu().numpy())
    y_true += list((y.detach().cpu().numpy()))

In [ ]:
print(accuracy_score(y_true, y_pred))
print(precision_score(y_true, y_pred))
print(recall_score(y_true, y_pred))
print(f1_score(y_true, y_pred))

### Experiments (10 points)

Experiment with the model and achieve better results. You can find advices [here](https://arxiv.org/abs/1801.06146). Implement and describe your experiments in details, mention what was helpful.

### 1. ?
### 2. ?
### 3. ?
Все эксперименты ниже, у них выводится номер и характеристики

In [ ]:
class RNNDropout(nn.Module):
    def __init__(self, hidden_dim, emb_dim, dropout):
        super().__init__()
        self.emb = nn.Embedding(len(TEXT.vocab), emb_dim)
        self.rnn_1 = nn.GRU(emb_dim, hidden_dim, batch_first=False)
        self.rnn_2 = nn.GRU(hidden_dim, hidden_dim, batch_first=False)
        self.rnn_3 = nn.GRU(hidden_dim, hidden_dim, batch_first=False)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(hidden_dim, 1)
            
    def forward(self, x):
        x = self.emb(x)    
        x = self.rnn_1(x)[0]
        x = self.dropout(x)
        x = self.rnn_2(x)[0]
        x = self.dropout(x)
        last_hidden = self.rnn_3(x)[1]

        # preds are outputs of linear layer BEFORE sigmoid
        preds = self.linear(last_hidden).reshape(-1)
        return preds

In [ ]:
em_sz = 200
nh = 300
dropout = 0.3
model = RNNDropout(nh, em_sz, dropout)
model.cuda()

In [ ]:
learning_rate = 1e-3
opt = optim.Adam(model.parameters(), lr=learning_rate)
loss_func = nn.BCEWithLogitsLoss()

In [ ]:
epochs = 15

In [ ]:
%%time

train_loss_history = []
valid_loss_history = []

train_loop('rnn_experiment_2')

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(np.arange(epochs) + 1, train_loss_history, label='train')
plt.plot(np.arange(epochs) + 1, valid_loss_history, label='valid')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# best score on validation
model.load_state_dict(torch.load('rnn_experiment_2_5.pt'))

In [ ]:
y_pred = []
y_true = []
model.eval()

for batch in test_iter:
    x = batch.text
    y = batch.label

    y_pred += list((torch.sigmoid(model(x)) >= 0.5).detach().cpu().numpy())
    y_true += list((y.detach().cpu().numpy()))

In [ ]:
print(accuracy_score(y_true, y_pred))
print(precision_score(y_true, y_pred))
print(recall_score(y_true, y_pred))
print(f1_score(y_true, y_pred))

In [ ]:
class RNNBatchNorm(nn.Module):
    def __init__(self, hidden_dim, emb_dim, dropout):
        super().__init__()
        self.emb = nn.Embedding(len(TEXT.vocab), emb_dim)
        self.rnn_1 = nn.GRU(emb_dim, hidden_dim, batch_first=False)
        self.rnn_2 = nn.GRU(hidden_dim, hidden_dim, batch_first=False)
        self.rnn_3 = nn.GRU(hidden_dim, hidden_dim, batch_first=False)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.batchnorm_1 = nn.BatchNorm1d(emb_dim)
        self.batchnorm_2 = nn.BatchNorm1d(hidden_dim)
        self.batchnorm_3 = nn.BatchNorm1d(hidden_dim)


        self.linear = nn.Linear(hidden_dim, 1)
            
    def forward(self, x):
        x = self.emb(x)
        # permute for batch_norm
        x = x.permute(1, 2, 0)
        x = self.batchnorm_1(x) 
        # inverse permute
        x = x.permute(2, 0, 1)
        
        x = self.rnn_1(x)[0]
        x = self.dropout(x)
        x = self.relu(x)
        x = x.permute(1, 2, 0)
        x = self.batchnorm_2(x)
        x = x.permute(2, 0, 1)


        x = self.rnn_2(x)[0]
        x = self.dropout(x)
        x = self.relu(x)
        x = x.permute(1, 2, 0)
        x = self.batchnorm_3(x)
        x = x.permute(2, 0, 1)

        last_hidden = self.rnn_3(x)[1]
        last_hidden = self.relu(last_hidden)
        last_hidden = self.dropout(last_hidden)

        # preds are outputs of linear layer BEFORE sigmoid
        preds = self.linear(last_hidden).reshape(-1)
        # make the batch second again
        return preds

In [ ]:
em_sz = 200
nh = 300
dropout = 0.3
model = RNNBatchNorm(nh, em_sz, dropout)
model.cuda()

In [ ]:
learning_rate = 1e-3
opt = optim.Adam(model.parameters(), lr=learning_rate)
loss_func = nn.BCEWithLogitsLoss()

In [ ]:
epochs = 20

In [ ]:
%%time

train_loss_history = []
valid_loss_history = []

train_loop('rnn_experiment_3')

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(np.arange(epochs) + 1, train_loss_history, label='train')
plt.plot(np.arange(epochs) + 1, valid_loss_history, label='valid')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# best score on validation
model.load_state_dict(torch.load('rnn_experiment_3_4.pt'))

In [ ]:
y_pred = []
y_true = []
model.eval()

for batch in test_iter:
    x = batch.text
    y = batch.label

    y_pred += list((torch.sigmoid(model(x)) >= 0.5).detach().cpu().numpy())
    y_true += list((y.detach().cpu().numpy()))

In [ ]:
print(accuracy_score(y_true, y_pred))
print(precision_score(y_true, y_pred))
print(recall_score(y_true, y_pred))
print(f1_score(y_true, y_pred))

Мне удалось повысить точность по сравнению с первым экспериментом. В целом эксперименты дали прибавку к точности и F1-оценке.
Кажется что, самым важным изменением было добавление нескольких слоев в rnn.